In [ ]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import csv
import numpy as np
import random
import pandas
import pickle
import glob
import os.path
import sys
import operator
import threading
from keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf

from sol import SOL

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
tf.test.is_gpu_available()

In [ ]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
#from models import ResearchModels
#from data import DataSet
import time
import os.path
import numpy
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix


In [ ]:
import keras
import numpy
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.wrappers import TimeDistributed


from keras.layers import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)

from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.models import Input


from tensorflow.keras import regularizers


from keras.layers import Activation
from sklearn.metrics import confusion_matrix


from tensorflow.python.util.tf_export import tf_export
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import array_ops

import kerastuner as kt
from kerastuner import HyperModel

from collections import deque
import sys

In [ ]:
"""

Functions for evaluation metrics

"""

def cm_tss(y_true,y_pred):
    cm = confusion_matrix(np.argmax(y_true,axis=1),np.argmax(y_pred,axis=1))
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2
    return cm,tss

######################################################################################

def cm_tss_binary(y_true,y_pred):
    cm = confusion_matrix(y_true,y_pred)
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2
    return cm,tss

######################################################################################

def compute_tss(TP,TN,FP,FN):
    tss = numpy.zeros(TP.shape[0])
    for i in range(TP.shape[0]):
        if TP[i]+FN[i] == 0.:
            if TP[i] == 0.:
                tss_aux1 = 0.  # float('NaN')
            else:
                tss_aux1 = -100 
        else:
            tss_aux1 = (TP[i] / (TP[i] + FN[i]))
            
        if FP[i]+TN[i] == 0.:
            if FP[i] == 0.:
                tss_aux2 = 0.  # float('NaN')
            else:
                tss_aux2 = -100 
        else:
            tss_aux2 = (FP[i] / (FP[i] + TN[i]))
        
        tss[i] = tss_aux1 - tss_aux2
    
    return tss

######################################################################################
# Correzione a posteriori per metrica myTP, myTN etc etc

def compute_tss_cat_ce(TP,TN,FP,FN,steps_per_epoch):
    TP=np.round(TP*steps_per_epoch)
    FP=np.round(FP*steps_per_epoch)
    TN=np.round(TN*steps_per_epoch)
    FN=np.round(FN*steps_per_epoch)
    tss = numpy.zeros(TP.shape[0])
    for i in range(TP.shape[0]):
        if TP[i]+FN[i] == 0.:
            if TP[i] == 0.:
                tss_aux1 = 0.  # float('NaN')
            else:
                tss_aux1 = -100 
        else:
            tss_aux1 = (TP[i] / (TP[i] + FN[i]))
            
        if FP[i]+TN[i] == 0.:
            if FP[i] == 0.:
                tss_aux2 = 0.  # float('NaN')
            else:
                tss_aux2 = -100 
        else:
            tss_aux2 = (FP[i] / (FP[i] + TN[i]))
        
        tss[i] = tss_aux1 - tss_aux2
    
    return tss

######################################################################################

def tss(y_true,y_pred):

    cm = confusion_matrix(numpy.argmax(y_true,axis=1),numpy.argmax(y_pred,axis=1))

    if cm.shape[0] == 1 and sum(y_true) == 0:

        a = 0.

        d = float(cm[0, 0])

        b = 0.

        c = 0.

    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:

        a = float(cm[0, 0])

        d = 0.

        b = 0.

        c = 0.

    elif cm.shape[0] == 2:

        a = float(cm[1, 1])

        d = float(cm[0, 0])

        b = float(cm[0, 1])

        c = float(cm[1, 0])

    TP = a

    TN = d

    FP = b

    FN = c



    if TP + FN == 0.:

        if TP == 0.:

            tss_aux1 = 0.  # float('NaN')

        else:

            tss_aux1 = -100  # float('Inf')

    else:

        tss_aux1 = (TP / (TP + FN))



    if (FP + TN) == 0.:

        if FP == 0.:

            tss_aux2 = 0.  # float('NaN')

        else:

            tss_aux2 = -100  # float('Inf')

    else:

        tss_aux2 = (FP / (FP + TN))



    tss = tss_aux1 - tss_aux2

    return tss

######################################################################################

def hss(y_true,y_pred):

    cm = confusion_matrix(numpy.argmax(y_true,axis=1),numpy.argmax(y_pred,axis=1))

    if cm.shape[0] == 1 and sum(y_true) == 0:

        a = 0.

        d = float(cm[0, 0])

        b = 0.

        c = 0.

    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:

        a = float(cm[0, 0])

        d = 0.

        b = 0.

        c = 0.

    elif cm.shape[0] == 2:

        a = float(cm[1, 1])

        d = float(cm[0, 0])

        b = float(cm[0, 1])

        c = float(cm[1, 0])

    TP = a

    TN = d

    FP = b

    FN = c



    if ((TP + FN) * (FN + TN) + (TP + FP) * (FP + TN)) == 0.:

        if (TP * TN - FN * FP) == 0:

            hss = 0.  # float('NaN')

        else:

            hss = -100  # float('Inf')

    else:

        hss = 2 * (TP * TN - FN * FP) / ((TP + FN) *

                                         (FN + TN) + (TP + FP) * (FP + TN))

    return hss





In [ ]:
# Functions for choosing the best threshold

def optimize_threshold_skill_scores(probability_prediction, Y_training):
    n_samples = 100
    step = 1. / n_samples
    
    xss = -1.
    xss_threshold = 0
    Y_best_predicted = numpy.zeros((Y_training.shape))
    tss_vector = numpy.zeros(n_samples)
    hss_vector = numpy.zeros(n_samples)
    xss_threshold_vector = numpy.zeros(n_samples)
    a = probability_prediction.max()
    b = probability_prediction.min()
    for threshold in range(1, n_samples):
        xss_threshold = step * threshold * numpy.abs(a - b) + b
        xss_threshold_vector[threshold] = xss_threshold
        Y_predicted = probability_prediction > xss_threshold
        res = metrics_classification(Y_training > 0, Y_predicted, print_skills=False)
        tss_vector[threshold] = res['tss']
        hss_vector[threshold] = res['hss']
        
    if numpy.max(tss_vector)==0.:
        max_tss = numpy.max(tss_vector) + numpy.finfo(float).eps
    else:
        max_tss = numpy.max(tss_vector)
        
    if numpy.max(hss_vector)==0.:
        max_hss = numpy.max(hss_vector) + numpy.finfo(float).eps
    else:
        max_hss = numpy.max(hss_vector)
    
            
    nss_vector = 0.5*((tss_vector/max_tss) + (hss_vector/max_hss))

    idx_best_nss = numpy.where(nss_vector==numpy.max(nss_vector))  
    print('idx best nss=',idx_best_nss)
    #best NSS
    best_xss_threshold = xss_threshold_vector[idx_best_nss]
    if len(best_xss_threshold)>1:
        best_xss_threshold = best_xss_threshold[0]
        Y_best_predicted = probability_prediction > best_xss_threshold
    else:
        Y_best_predicted = probability_prediction > best_xss_threshold
    print('best NSS')
    metrics_training = metrics_classification(Y_training > 0, Y_best_predicted)
    
    #best TSS
    idx_best_tss = numpy.where(tss_vector==numpy.max(tss_vector))  
    print('idx best tss=',idx_best_tss)
    best_xss_threshold_tss = xss_threshold_vector[idx_best_tss]
    if len(best_xss_threshold_tss)>1:
        best_xss_threshold_tss = best_xss_threshold_tss[0]
        Y_best_predicted_tss = probability_prediction > best_xss_threshold_tss
    else:
        Y_best_predicted_tss = probability_prediction > best_xss_threshold_tss
    print('best TSS')
    metrics_training_tss = metrics_classification(Y_training > 0, Y_best_predicted_tss)
    
    #best HSS
    idx_best_hss = numpy.where(hss_vector==numpy.max(hss_vector))  
    print('idx best hss=',idx_best_hss)
    best_xss_threshold_hss = xss_threshold_vector[idx_best_hss]
    if len(best_xss_threshold_hss)>1:
        best_xss_threshold_hss = best_xss_threshold_hss[0]
        Y_best_predicted_hss = probability_prediction > best_xss_threshold_hss
    else:
        Y_best_predicted_hss = probability_prediction > best_xss_threshold_hss
    print('best HSS')
    metrics_training_hss = metrics_classification(Y_training > 0, Y_best_predicted_hss)
    
    
    
    #best (TSS+HSS)/2
    comb_tss_hss = (hss_vector+tss_vector)/2.
    idx_best_tss_hss = numpy.where(comb_tss_hss==numpy.max(comb_tss_hss)) 
    print('idx best (tss+hss)/2 =',idx_best_tss_hss)
    best_xss_threshold_tss_hss = xss_threshold_vector[idx_best_tss_hss]
    if len(best_xss_threshold_tss_hss)>1:
        best_xss_threshold_tss_hss = best_xss_threshold_tss_hss[0]
        Y_best_predicted_tss_hss = probability_prediction > best_xss_threshold_tss_hss
    else:
        Y_best_predicted_tss_hss = probability_prediction > best_xss_threshold_tss_hss
    print('best (TSS+HSS)/2')
    metrics_training_tss_hss = metrics_classification(Y_training > 0, Y_best_predicted_tss_hss)
    
   

    return best_xss_threshold, metrics_training, nss_vector, best_xss_threshold_tss, best_xss_threshold_hss,best_xss_threshold_tss_hss

def metrics_classification(y_real, y_pred, print_skills=True):

    cm, far, pod, acc, hss, tss, fnfp, csi = classification_skills(y_real, y_pred)

    if print_skills:
        print ('confusion matrix')
        print (cm)
        print ('false alarm ratio       \t', far)
        print ('probability of detection\t', pod)
        print ('accuracy                \t', acc)
        print ('hss                     \t', hss)
        print ('tss                     \t', tss)
        print ('balance                 \t', fnfp)
        print ('csi                 \t', csi)

    balance_label = float(sum(y_real)) / y_real.shape[0]

    #cm, far, pod, acc, hss, tss, fnfp = classification_skills(y_real, y_pred)

    return {
        "cm": cm,
        "far": far,
        "pod": pod,
        "acc": acc,
        "hss": hss,
        "tss": tss,
        "fnfp": fnfp,
        "balance label": balance_label,
        "csi": csi}

def classification_skills(y_real, y_pred):

    cm = confusion_matrix(y_real, y_pred)

    if cm.shape[0] == 1 and sum(y_real) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_real) == y_real.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if (TP + FP + FN + TN) == 0.:
        if (TP + TN) == 0.:
            acc = 0.  # float('NaN')
        else:
            acc = -100  # float('Inf')
    else:
        acc = (TP + TN) / (TP + FP + FN + TN)

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2

    if ((TP + FN) * (FN + TN) + (TP + FP) * (FP + TN)) == 0.:
        if (TP * TN - FN * FP) == 0:
            hss = 0.  # float('NaN')
        else:
            hss = -100  # float('Inf')
    else:
        hss = 2 * (TP * TN - FN * FP) / ((TP + FN) *
                                         (FN + TN) + (TP + FP) * (FP + TN))

    if FP == 0.:
        if FN == 0.:
            fnfp = 0.  # float('NaN')
        else:
            fnfp = -100  # float('Inf')
    else:
        fnfp = FN / FP

    if (TP + FN) == 0.:
        if TP == 0.:
            pod = 0  # float('NaN')
        else:
            pod = -100  # float('Inf')
    else:
        pod = TP / (TP + FN)


    if (TP + FP) == 0.:
        if FP == 0.:
            far = 0.  # float('NaN')
        else:
            far = -100  # float('Inf')
    else:
        far = FP / (TP + FP)

    #acc = (a + d) / (a + b + c + d)
    #tpr = a / (a + b)
    #tnr = d / (d + c)
    #wtpr = a / (a + b) * (a + c) / (a + b + c + d) + d / (c + d) * (b + d) / (a + b + c + d)
    #pacc = a / (a + c)
    #nacc = d / (b + d)
    #wacc = a / (a + c) * (a + c) / (a + b + c + d) + d / (b + d) * (b + d) / (a + b + c + d)

    # if the cm has a row or a column equal to 0, we have bad tss
    if TP+FN == 0 or TN+FP == 0 or TP+FP == 0 or TN+FN == 0:
        tss = 0
    if TP+FP+FN==0:
        csi = 0
    else:
        csi = TP/(TP+FP+FN)

    #, pod, acc, hss, tss, fnfp, tpr, tnr, pacc, nacc, wacc, wtpr)
    return cm.tolist(), far, pod, acc, hss, tss, fnfp, csi

def compute_cm_tss_threshold(y, pred,threshold):
    # Compute confusion matrix and TSS given y_true and y_pred in categorical form
    pred_threshold = pred > threshold
    cm = confusion_matrix(y,pred_threshold)
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2
    
    if ((TP + FN) * (FN + TN) + (TP + FP) * (FP + TN)) == 0.:
        if (TP * TN - FN * FP) == 0:
            hss = 0.  # float('NaN')
        else:
            hss = -100  # float('Inf')
    else:
        hss = 2 * (TP * TN - FN * FP) / ((TP + FN) *
                                         (FN + TN) + (TP + FP) * (FP + TN))
    
    if TP+FP+FN==0:
        CSI = 0
    else:
        CSI = TP/(TP+FP+FN)

    
    return cm, tss, hss, CSI
        
    
def compute_cm_tss(y, pred):
    # Compute confusion matrix and TSS given y_true and y_pred in categorical form
    cm = confusion_matrix(y,pred)
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2
    
    if ((TP + FN) * (FN + TN) + (TP + FP) * (FP + TN)) == 0.:
        if (TP * TN - FN * FP) == 0:
            hss = 0.  # float('NaN')
        else:
            hss = -100  # float('Inf')
    else:
        hss = 2 * (TP * TN - FN * FP) / ((TP + FN) *
                                         (FN + TN) + (TP + FP) * (FP + TN))
    
    if TP+FP+FN==0:
        CSI = 0
    else:
        CSI = TP/(TP+FP+FN)

    
    return cm, tss, hss, CSI


In [ ]:
"""

General functions

"""

def divide(a, b):
    if b == 0:
        return np.nan
    else: 
        return a/b

acc = lambda TN,FP,FN,TP: np.nan_to_num(divide(TP+TN,TN+FP+FN+TP))
prec = lambda TN,FP,FN,TP: np.nan_to_num(divide(TP,FP+TP))
rec = lambda TN,FP,FN,TP: np.nan_to_num(divide(TP,FN+TP))
spec = lambda TN,FP,FN,TP: np.nan_to_num(divide(TN,FP+TN))
f1s = lambda TN,FP,FN,TP: 2.*np.nan_to_num(divide(prec(TN,FP,FN,TP)*\
                          rec(TN,FP,FN,TP),prec(TN,FP,FN,TP)+rec(TN,FP,FN,TP)))
tss = lambda TN,FP,FN,TP: rec(TN,FP,FN,TP)+spec(TN,FP,FN,TP)-1.
csi = lambda TN,FP,FN,TP: np.nan_to_num(divide(TP,FN+FP+TP))
hss1 = lambda TN,FP,FN,TP: np.nan_to_num(divide(TP-FP,FN+TP))
hss2 = lambda TN,FP,FN,TP: np.nan_to_num(divide((2.*((TP*TN)-(FP*FN))),((TP+FN)\
                            *(FN+TN))+((TP+FP)*(TN+FP))))

######################################################################################

def cm_score_binary(y_true,y_pred,thresh,score):
    y_pred = np.array([0 if y_pred[i]<thresh else 1 for i in range(0,len(y_pred))])
    cm = confusion_matrix(y_true,y_pred)
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c
    
    if score == 'accuracy':
        score = acc
    if score == 'precision':
        score = prec
    if score == 'recall':
        score = rec    
    if score == 'specificity':
        score = spec        
    if score == 'f1_score':
        score = f1s
    if score == 'tss':
        score = tss
    if score == 'csi':
        score = csi
    if score == 'hss1':
        score = hss1        
    if score == 'hss2':
        score = hss2 
    


    return cm,score(TN,FP,FN,TP)

def score_training(y_true,y_pred):
    
    thresh = 0.5
    score = 'tss'
    
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()
    
    y_pred = np.array([0 if y_pred[i]<thresh else 1 for i in range(0,len(y_pred))])
    cm = confusion_matrix(y_true,y_pred)

    
    if cm.shape[0] == 1 and sum(y_true) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_true) == y_true.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c
    
    if score == 'accuracy':
        score = acc
    if score == 'precision':
        score = prec
    if score == 'recall':
        score = rec    
    if score == 'specificity':
        score = spec        
    if score == 'f1_score':
        score = f1s
    if score == 'tss':
        score = tss
    if score == 'csi':
        score = csi
    if score == 'hss1':
        score = hss1        
    if score == 'hss2':
        score = hss2 

    return score(TN,FP,FN,TP)


In [ ]:
#set the variables of input shape

#number of images in time series
seq_length = 40
#size of images
number_channels = 1
image_shape = (128,128,number_channels)


In [ ]:
#LRCN MODEL

# the input shape is composed by (number of images in time series, image height, image width, number of channels)

#in the case no multi channel data are used set number_of_channels equal to 1

input_shape = (seq_length, image_shape[0], image_shape[1], number_channels)

initialiser = 'glorot_uniform'

reg_lambda  = 0.01

model = Sequential()


# first block
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same',kernel_regularizer=regularizers.l2(reg_lambda),
                                               kernel_initializer=initialiser) , 
                                  input_shape=input_shape)) 
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

# second block
model.add(TimeDistributed(Conv2D(32, (5, 5), strides=(2, 2), padding='same',kernel_regularizer=regularizers.l2(reg_lambda),
                                               kernel_initializer=initialiser), 
                                  input_shape=self.input_shape)) 
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

# third block
model.add(TimeDistributed(Conv2D(32, (3, 3), strides=(2, 2), padding='same',kernel_regularizer=regularizers.l2(reg_lambda),
                                               kernel_initializer=initialiser), 
                                  input_shape=self.input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

# fourth block
model.add(TimeDistributed(Conv2D(32, (3, 3), strides=(2, 2), padding='same',kernel_regularizer=regularizers.l2(reg_lambda),
                                               kernel_initializer=initialiser), 
                                  input_shape=self.input_shape)) 
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
        
        
# MLP
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(64,activation='relu')))
model.add(TimeDistributed(Dropout(0.1)))

#LSTM
model.add(LSTM(50, return_sequences=False, dropout=0.5)) #50 100 256

#output
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-4, decay=1e-6)

loss = SOL(score='tss',distribution='uniform', mu=0.5, delta=0.1)

metrics = [score_training]

model.compile(loss=loss, optimizer=optimizer,metrics=metrics, run_eagerly=True)

print(model.summary())



In [ ]:

import sklearn
from keras.callbacks import LearningRateScheduler
def decay_schedule(epoch, lr):
    # decay by 0.1 every 5 epochs; use `% 1` to decay after each epoch
    if (epoch % 10 == 0) and (epoch != 0):
        lr = lr / 2.
    return lr

# Function for training the deep neural network
def train_uniform(model, seq_length, model_name,data_type, X_train_file,y_train_file,X_val_file, y_val_file,
                   saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=True, batch_size=32, nb_epoch=100,cl='C',fold=1):

    checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints', model + '-' + data_type + '-'+\
                              '_tss_loss_ten_fold_uniform_classes_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4_'+str(batch_size)+'batch_'+str(nb_epoch)+'epoch' \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=False) #
   
    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(patience=1000)
    lr_scheduler = LearningRateScheduler(decay_schedule)
    # Helper: Save results.
    timestamp = time.time()
    csv_logger = CSVLogger(os.path.join('data', 'logs', model + '-' + 'training-' + \
        str(timestamp) + '.log'))
    

    X_train = np.load(X_train_file)/4000.
    y_train = np.load(y_train_file).astype(numpy.float32)
    
    X_val = np.load(X_val_file)/4000.
    y_val = np.load(y_val_file).astype(numpy.float32)
  
    X_train,y_train = sklearn.utils.shuffle(X_train,y_train,random_state=42)
    X_train,y_train = sklearn.utils.shuffle(X_train,y_train,random_state=42)
    X_train,y_train = sklearn.utils.shuffle(X_train,y_train,random_state=42)
    
    X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)
    X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)
    X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)

    dim_y_train=y_train.shape[0]
    
    dim_y_val=y_val.shape[0]
    
    steps_per_epoch = dim_y_train//batch_size 
    validation_step = dim_y_val//batch_size

    model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
        callbacks=[ early_stopper, csv_logger, checkpointer],#[early_stopper], #tb,, checkpointer csvlogger
        epochs=nb_epoch)

    
    # plot the training loss and accuracy
    rm.model.save('data/'+model+'_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+\
                  '_tss_loss_ten_fold_uniform_classes_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4.hdf5') #_weight_class
    

    a_file = open('data/'+model+'_history_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+\
                  '_tss_loss_ten_fold_uniform_classes_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4.pkl', "wb")
    pickle.dump(rm.model.history.history, a_file)
    a_file.close()
    

    
    return model #, metrics.get_data()



In [ ]:

model_name = 'lrcn_sol' 
saved_model = None  # None or weights file
class_limit = None  # int, can be 1-101 or None

load_to_memory = True  # pre-load the sequences into memory
batch_size = 128
nb_epoch = 100 
data_type = 'images'


cl='C'
folder_data='data/'
fold=1

training_key = False

#Train the deep neaural network given a training and validation set
if training:
    # Filenames of the training data
    X_train_file = folder_data+'X_train_aug_fold_'+str(fold)+'.npy'#
    y_train_file = folder_data+'Y_train_aug_'+cl+'_discard_fold_'+str(fold)+'.npy'

    # Filenames of the validation data
    X_val_file = folder_data+'X_validation_aug_discard_fold_'+str(fold)+'.npy'
    y_val_file = folder_data+'Y_validation_aug_'+cl+'_discard_fold_'+str(fold)+'.npy'

    model_history=train_uniform(model, seq_length, model_name,data_type, X_train_file,y_train_file,X_val_file, y_val_file,
                    saved_model=saved_model,
          class_limit=class_limit, image_shape=image_shape,
          load_to_memory=load_to_memory, batch_size=batch_size, nb_epoch=nb_epoch,cl=cl,fold=fold)

    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0,len(model_history.history.history["loss"])), model_history.history.history["loss"], label="train_loss") #nb_epoch
    plt.plot(np.arange(0, len(model_history.history.history["val_loss"])), model_history.history.history["val_loss"], label="val_loss")
    plt.title("Loss on Dataset") #Training  and Accuracy
    plt.xlabel("Epoch #")
    plt.ylabel("Loss") #/Accuracy
    plt.legend(loc="upper right")
    plt.savefig('plot_'+model+'_loss_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+\
            '_tss_loss_ten_fold_uniform_classes_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4.png')

    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0,len(model_history.history.history["score_training"])), model_history.history.history["score_training"], label="train_score") #nb_epoch
    plt.plot(np.arange(0, len(model_history.history.history["val_score_training"])), model_history.history.history["val_score_training"], label="val_score")
    plt.title("Score on Dataset") #Training  and Accuracy
    plt.xlabel("Epoch #")
    plt.ylabel("Score") #/Accuracy
    plt.legend(loc="upper right")
    plt.savefig('plot_'+model+'_score_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+\
            '_tss_loss_ten_fold_uniform_classes_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4.png')




In [ ]:
# Read the validation and test data
import gc
import sklearn

cl='C'
fold=1
folder_data='data/'

# Filenames of validation and test data
X_test_file = folder_data+'X_test_discard_fold_'+str(fold)+'.npy'
y_test_file = folder_data+'Y_test_'+cl+'_discard_fold_'+str(fold)+'.npy'

X_test = np.load(X_test_file)/4000.
y_test = np.load(y_test_file).astype(np.float32)



X_val_file = folder_data+'X_validation_aug_discard_fold_'+str(fold)+'.npy'
y_val_file = folder_data+'Y_validation_aug_'+cl+'_discard_fold_'+str(fold)+'.npy'
  
X_val = np.load(X_val_file)/4000.
y_val = np.load(y_val_file).astype(np.float32)

X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)
X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)
X_val,y_val = sklearn.utils.shuffle(X_val,y_val,random_state=42)


In [ ]:
# Choose the best model accordingly to the highest TSS on the validation set and predict on the test set

from pandas import read_pickle
mod_hist=read_pickle('lrcn_sol_history_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+'_tss_loss_ten_fold_uniform_all_classes_no_aug_discard_24h_'+cl+'_'+str(fold)+'_lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4.pkl')

idx =np.where(mod_hist["val_score_training"]==np.max(mod_hist["val_score_training"]))

print(idx)
print(np.array(mod_hist["val_score_training"])[idx])

folder = 'checkpoints/'

list_epochs = sorted(glob.glob(folder+'lrcn_sol-images-_tss_loss_ten_fold_uniform_all_classes_no_aug_discard_24h_'+cl+'_'+str(fold)+'_'+\
                               'lrcn_32kern_7_5_3_3_size_reg_dense_64_lstm_50_adam_lr_1e-4_'+str(batch_size)+'batch_'+str(nb_epoch)+'epoch*.hdf5'))
print(len(list_epochs))

model = load_model(list_epochs[idx[0][0]],compile=False)
print(list_epochs[idx[0][0]])


pred_test = model.predict(X_test)
pred_prob_test = pred_test[:,0]#if use binary ce, tss loss..

pred_prob_test = pred_prob_test.reshape(1,len(pred_prob_test))
pred_prob_test = pred_prob_test[0]
pred_0_1 = pred_prob_test > 0.5

print('Case epoch with maximum tss_val and threshold=0.5')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, pred_0_1)#_weight
print('Skill scores (threshold=0.5)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

#####################################################################
#choose the best threshold in validation with the best epoch 

pred_val = model.predict(X_val)
gc.collect()

pred_prob_val = pred_val[:,0]#if use binary crossentropy, tss loss ...

#OPTIMIZE NSS, TSS, HSS, ..
threshold_nss, metrics_training, nss_vector, threshold_tss, threshold_hss, threshold_tss_hss = \
                                     optimize_threshold_skill_scores(pred_prob_val, y_val)
pred_0_1_opt_tss = pred_prob_test > threshold_tss

print('Case epoch with maximum tss_val and optimum threshold')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, pred_0_1_opt_tss)#_weight
print('Skill scores (opt threshold)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

pred_0_1_test = pred_prob_test > 0.5

print('Case epoch with maximum tss_val and threshold = 0.5')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, pred_0_1_test)#_weight
print('Skill scores (threshold = 0.5)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))